In [76]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime as dt

# Load the dataset
data = pd.read_csv("./data/train.csv", )

# Features (input) and targets (output)
data['date'] = data.apply(lambda row: f"{int(row['year'])}-{int(row['month'])}-{int(row['day'])} {int(row['hour'])}:{int(row['min'])}:{row['sec']/100}", axis=1)
data['date'] = pd.to_datetime(data['date'])

X = data[['date', 'lat', 'lon', 'depth', 'class']].values
y = data[['year_as','month_as', 'day_as', 'hour_as', 'min_as', 'sec_as', 'lat_as', 'lon_as', 'depth_as', 'class_as']]

data

,Unnamed: 0,year,month,day,hour,min,sec,lat,lon,depth,...,month_as,day_as,hour_as,min_as,sec_as,lat_as,lon_as,depth_as,class_as,date
0,0,1980,2,15,9,9,350,40.54,78.31,-1,...,2,15,9,45,540,40.45,78.24,-1,7.7,1980-02-15 09:09:03.500
1,1,1980,3,16,1,44,30,41.42,75.40,-1,...,3,25,10,58,236,41.40,75.42,-1,8.7,1980-03-16 01:44:00.300
2,2,1980,7,5,20,25,220,41.46,77.30,20,...,7,5,21,41,335,41.56,77.25,5,7.3,1980-07-05 20:25:02.200
3,3,1980,8,31,23,38,372,40.52,77.46,-1,...,9,4,22,56,92,40.51,77.37,-1,7.3,1980-08-31 23:38:03.720
4,4,1980,9,4,6,47,166,44.16,79.08,10,...,9,14,9,16,160,44.18,79.30,-1,7.2,1980-09-04 06:47:01.660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,1245,2013,12,12,0,46,471,44.12,81.18,10,...,12,12,6,1,159,44.14,81.42,5,6.2,2013-12-12 00:46:04.710
1246,1246,2013,12,18,1,44,283,41.00,78.45,15,...,12,24,10,6,388,41.04,78.44,10,7.5,2013-12-18 01:44:02.830
1247,1247,2013,12,21,14,21,464,37.12,71.31,0,...,12,22,1,24,74,37.23,71.17,5,9.3,2013-12-21 14:21:04.640
1248,1248,2013,12,22,7,45,314,38.22,72.07,20,...,12,26,8,26,227,38.47,72.14,0,7.7,2013-12-22 07:45:03.140


In [77]:
training_df = pd.DataFrame()
training_df[['date', 'lat', 'lon', 'depth', 'class']] = X
training_df

,date,lat,lon,depth,class
0,1980-02-15 09:09:03.500,40.54,78.31,-1,12.0
1,1980-03-16 01:44:00.300,41.42,75.4,-1,10.4
2,1980-07-05 20:25:02.200,41.46,77.3,20,13.6
3,1980-08-31 23:38:03.720,40.52,77.46,-1,11.9
4,1980-09-04 06:47:01.660,44.16,79.08,10,12.4
...,...,...,...,...,...
1245,2013-12-12 00:46:04.710,44.12,81.18,10,10.9
1246,2013-12-18 01:44:02.830,41.0,78.45,15,10.5
1247,2013-12-21 14:21:04.640,37.12,71.31,0,11.5
1248,2013-12-22 07:45:03.140,38.22,72.07,20,10.1


In [78]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

training_df['item_id'] = 1

train_data = TimeSeriesDataFrame(
    training_df,
    id_column="item_id",
    timestamp_column="date")

train_data.head()

lat    lon depth class
item_id timestamp                                        
1       1980-02-15 09:09:03.500  40.54  78.31    -1  12.0
        1980-03-16 01:44:00.300  41.42   75.4    -1  10.4
        1980-07-05 20:25:02.200  41.46   77.3    20  13.6
        1980-08-31 23:38:03.720  40.52  77.46    -1  11.9
        1980-09-04 06:47:01.660  44.16  79.08    10  12.4

In [79]:
predictors = []
features_to_predict = ['year_as','month_as', 'day_as', 'hour_as', 'min_as', 'sec_as', 'lat_as', 'lon_as', 'depth_as', 'class_as']

for i in range(len(features_to_predict)):

    training_df[features_to_predict[i]]=y[features_to_predict[i]]
    
    train_data = TimeSeriesDataFrame(
    training_df,
    id_column="item_id",
    timestamp_column="date")

    predictor = TimeSeriesPredictor(target=features_to_predict[i], prediction_length=1, eval_metric="mae", freq='D')

    predictor.fit(
        train_data,
        presets="best_quality",
        time_limit=600
    )

    predictors.append(predictor)

Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to 'c:\Users\Tora\Downloads\kbtu-data-science-challenge-2025-entry-task-new\Final\AutogluonModels\ag-20250412_191326'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.6
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
GPU Count:          1
Memory Avail:       15.28 GB / 63.92 GB (23.9%)
Disk Space Avail:   122.37 GB / 1217.02 GB (10.1%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAE,
 'freq': 'D',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 1,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'year_as',
 'time_limit': 600,
 'verbosity': 2}

train_data wi

Provided train_data has 12367 rows (NaN fraction=92.1%), 1 time series. Median time series length is 12367 (min=12367, max=12367). 

Provided data contains following columns:
	target: 'year_as'
	past_covariates:
		categorical:        ['lat', 'lon', 'depth', 'class']
		continuous (float): []

To learn how to fix incorrectly inferred types, please see documentation for TimeSeriesPredictor.fit

AutoGluon will gauge predictive performance using evaluation metric: 'MAE'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2025-04-13 00:13:28
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'ChronosZeroShot[bolt_base]', 'ChronosFineTuned[bolt_small]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST', 'TiDE']
Training timeseries model SeasonalNaive. Training for up to 46.0s of the 597.7s of rema

In [80]:
for predictor in predictors:
    print(predictor.model_best)
    print(predictor.fit_summary())

SeasonalNaive
****************** Summary of fit() ******************
Estimated performance of each model:
                   model  score_val  pred_time_val  fit_time_marginal  \
0          SeasonalNaive        NaN       1.934154           2.730077   
1       RecursiveTabular        NaN       0.129923           5.285005   
2                   NPTS        NaN       2.278809           2.000262   
3  DynamicOptimizedTheta        NaN      27.326314          28.327689   
4          DirectTabular        NaN       0.258320           4.592027   
5                AutoETS        NaN       4.631690           4.924231   

   fit_order  
0          1  
1          2  
2          4  
3          5  
4          3  
5          6  
Number of models trained: 6
Types of models trained:
{'MultiWindowBacktestingModel'}
****************** End of fit() summary ******************
{'model_types': {'SeasonalNaive': 'MultiWindowBacktestingModel', 'RecursiveTabular': 'MultiWindowBacktestingModel', 'DirectTabular': 

In [91]:
data_t = pd.read_csv("./data/test.csv", )

# Features (input) and targets (output)
data_t['date'] = data_t.apply(lambda row: f"{int(row['year'])}-{int(row['month'])}-{int(row['day'])} {int(row['hour'])}:{int(row['min'])}:{row['sec']/100}", axis=1)
data_t['date'] = pd.to_datetime(data_t['date'])

X = data_t[['date', 'lat', 'lon', 'depth', 'class']].values

data_t

,id_eq,year,month,day,hour,min,sec,lat,lon,depth,class,date
0,996,2018,2,14,23,21,505,38.46,73.45,60,10.7,2018-02-14 23:21:05.050
1,1336,2020,10,2,7,16,418,37.22,71.06,20,10.2,2020-10-02 07:16:04.180
2,730,2016,4,15,15,21,269,51.05,89.48,10,10.9,2016-04-15 15:21:02.690
3,696,2016,1,26,23,19,156,37.18,70.45,0,11.5,2016-01-26 23:19:01.560
4,1460,2021,10,26,17,47,43,42.27,83.46,10,11.0,2021-10-26 17:47:00.430
...,...,...,...,...,...,...,...,...,...,...,...,...
295,1422,2021,7,10,2,14,421,38.44,70.27,10,13.4,2021-07-10 02:14:04.210
296,737,2016,4,27,10,54,2,37.53,71.42,25,10.2,2016-04-27 10:54:00.020
297,1737,2023,8,21,4,40,104,37.26,71.29,5,10.7,2023-08-21 04:40:01.040
298,1184,2019,9,29,22,47,449,37.15,71.00,10,10.3,2019-09-29 22:47:04.490


In [92]:
test_df = pd.DataFrame()
test_df[['date', 'lat', 'lon', 'depth', 'class']] = X
test_df

,date,lat,lon,depth,class
0,2018-02-14 23:21:05.050,38.46,73.45,60,10.7
1,2020-10-02 07:16:04.180,37.22,71.06,20,10.2
2,2016-04-15 15:21:02.690,51.05,89.48,10,10.9
3,2016-01-26 23:19:01.560,37.18,70.45,0,11.5
4,2021-10-26 17:47:00.430,42.27,83.46,10,11.0
...,...,...,...,...,...
295,2021-07-10 02:14:04.210,38.44,70.27,10,13.4
296,2016-04-27 10:54:00.020,37.53,71.42,25,10.2
297,2023-08-21 04:40:01.040,37.26,71.29,5,10.7
298,2019-09-29 22:47:04.490,37.15,71.0,10,10.3


In [93]:
test_df['item_id'] = 1

In [94]:
for i in range(len(predictors)):
    test_df[features_to_predict[i]]=np.nan
    test_data = TimeSeriesDataFrame(
        test_df,
        id_column="item_id",
        timestamp_column="date")
    predictions = predictors[i].predict(test_data)
    print(predictions.head())

data with frequency 'None' has been resampled to frequency 'D'.


Model not specified in predict, will default to the model with the best validation score: SeasonalNaive
data with frequency 'None' has been resampled to frequency 'D'.


                      mean     0.1     0.2     0.3     0.4     0.5     0.6  \
item_id timestamp                                                            
1       2023-10-16  1998.0  1984.5  1988.0  1991.0  1994.0  1998.0  2004.0   

                       0.7     0.8     0.9  
item_id timestamp                           
1       2023-10-16  2009.0  2011.0  2013.0  


Model not specified in predict, will default to the model with the best validation score: SeasonalNaive
data with frequency 'None' has been resampled to frequency 'D'.


                    mean  0.1  0.2  0.3  0.4  0.5  0.6  0.7   0.8   0.9
item_id timestamp                                                      
1       2023-10-16   7.0  2.0  3.0  4.0  6.0  7.0  8.0  9.0  10.0  11.0


...

                     mean    0.1    0.2    0.3    0.4    0.5    0.6     0.7  \
item_id timestamp                                                             
1       2023-10-16  77.16  71.35  73.03  74.51  75.47  77.16  77.42  78.215   

                      0.8     0.9  
item_id timestamp                  
1       2023-10-16  79.13  80.295  


Model not specified in predict, will default to the model with the best validation score: SeasonalNaive
data with frequency 'None' has been resampled to frequency 'D'.


                    mean  0.1  0.2  0.3  0.4  0.5  0.6  0.7   0.8   0.9
item_id timestamp                                                      
1       2023-10-16   0.0 -1.0 -1.0 -1.0 -1.0  0.0  5.0  8.0  10.0  15.0


Model not specified in predict, will default to the model with the best validation score: SeasonalNaive


                    mean  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9
item_id timestamp                                                    
1       2023-10-16   7.9  6.7  7.1  7.4  7.6  7.9  8.3  8.6  9.2  9.7
